In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/19981126-tp/X_test.csv
/kaggle/input/19981126-tp/y_train.csv
/kaggle/input/19981126-tp/X_train.csv
/kaggle/input/19981126-tp/submit_sample.csv


In [2]:
# torch library import
import torch
import torch.optim as optim
import torch.nn as nn


# GPU usage
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# data loading
x_train = pd.read_csv("/kaggle/input/19981126-tp/X_train.csv")
y_train = pd.read_csv("/kaggle/input/19981126-tp/y_train.csv")
x_test = pd.read_csv("/kaggle/input/19981126-tp/X_test.csv")
submit = pd.read_csv("/kaggle/input/19981126-tp/submit_sample.csv")

# insight
x_train.head()

,지점,일시,평균 풍속(m/s),평균기압(hPa),평균 상대습도(%),평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec),시도명
0,22190,2021-05-04,NaN,1010.8,NaN,NaN,16.0,2.6,1.6,2.2,4.0,6.3,9.1,광주
1,22189,2021-03-10,7.6,1028.3,59.0,11.3,15.3,2.2,1.4,2.0,3.3,5.5,9.1,부산
2,22105,2021-08-12,2.0,1010.8,82.0,27.4,27.1,2.9,1.8,2.3,3.9,9.2,10.7,대구
3,22105,2021-02-02,10.8,1018.3,43.0,2.6,13.3,4.6,2.8,3.6,6.0,7.9,10.7,세종
4,22105,2021-06-16,7.0,1008.1,86.0,21.0,20.2,1.9,1.1,1.4,2.4,5.1,5.8,서울


In [4]:
# check nulls, dtypes
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12777 entries, 0 to 12776
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   지점           12777 non-null  int64  
 1   일시           12777 non-null  object 
 2   평균 풍속(m/s)   11341 non-null  float64
 3   평균기압(hPa)    12731 non-null  float64
 4   평균 상대습도(%)   12179 non-null  float64
 5   평균 기온(°C)    12285 non-null  float64
 6   평균 수온(°C)    12388 non-null  float64
 7   평균 최대 파고(m)  12661 non-null  float64
 8   평균 유의 파고(m)  12528 non-null  float64
 9   최고 유의 파고(m)  12754 non-null  float64
 10  최고 최대 파고(m)  12754 non-null  float64
 11  평균 파주기(sec)  12665 non-null  float64
 12  최고 파주기(sec)  12753 non-null  float64
 13  시도명          12777 non-null  object 
dtypes: float64(11), int64(1), object(2)
memory usage: 1.4+ MB


In [5]:
x_train['시도명'].unique()

array(['광주', '부산', '대구', '세종', '서울', '울산', '경기', '대전', '전남', '인천', '강원',
       '전북', '충북'], dtype=object)

In [6]:
# object label preprocessing
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cities = pd.concat([x_train['시도명'], x_test['시도명']], axis = 0)
le.fit(cities)

x_train['시도명'] = le.transform(x_train['시도명'])
x_test['시도명'] = le.transform(x_test['시도명'])

x_test['시도명'].unique()

array([ 2,  3,  5,  4,  1, 10,  6,  7,  0,  8,  9, 12, 11])

In [7]:
# drop column
# 일반적으로 최대파고=1.61×유의파고의 관계가 있다.
x_train = x_train.drop(["최고 유의 파고(m)"], axis = 1)
x_test = x_test.drop(["최고 유의 파고(m)"], axis = 1)

In [8]:
# check nulls, dtypes
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12777 entries, 0 to 12776
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   지점           12777 non-null  int64  
 1   일시           12777 non-null  object 
 2   평균 풍속(m/s)   11341 non-null  float64
 3   평균기압(hPa)    12731 non-null  float64
 4   평균 상대습도(%)   12179 non-null  float64
 5   평균 기온(°C)    12285 non-null  float64
 6   평균 수온(°C)    12388 non-null  float64
 7   평균 최대 파고(m)  12661 non-null  float64
 8   평균 유의 파고(m)  12528 non-null  float64
 9   최고 최대 파고(m)  12754 non-null  float64
 10  평균 파주기(sec)  12665 non-null  float64
 11  최고 파주기(sec)  12753 non-null  float64
 12  시도명          12777 non-null  int64  
dtypes: float64(10), int64(2), object(1)
memory usage: 1.3+ MB


In [9]:
# nan value preprocessing
nan_columns = ['평균 풍속(m/s)', '평균기압(hPa)',
               '평균 상대습도(%)', '평균 기온(°C)',
               '평균 수온(°C)', '평균 최대 파고(m)',
               '평균 유의 파고(m)', '최고 최대 파고(m)',
               '평균 파주기(sec)', '최고 파주기(sec)']

for col in nan_columns:
    x_train[col] = x_train[col].fillna(x_train[col].mean())
    x_test[col] = x_test[col].fillna(x_test[col].mean())

In [10]:
# date to month preprocessing (ex. 2019-09-06 -> 9)
def date_to_month(date):
    ar = date.split('-')
    month = int(ar[1])
    return month

x_train['일시'] = x_train['일시'].apply(date_to_month)
x_test['일시'] = x_test['일시'].apply(date_to_month)

In [11]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12777 entries, 0 to 12776
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   지점           12777 non-null  int64  
 1   일시           12777 non-null  int64  
 2   평균 풍속(m/s)   12777 non-null  float64
 3   평균기압(hPa)    12777 non-null  float64
 4   평균 상대습도(%)   12777 non-null  float64
 5   평균 기온(°C)    12777 non-null  float64
 6   평균 수온(°C)    12777 non-null  float64
 7   평균 최대 파고(m)  12777 non-null  float64
 8   평균 유의 파고(m)  12777 non-null  float64
 9   최고 최대 파고(m)  12777 non-null  float64
 10  평균 파주기(sec)  12777 non-null  float64
 11  최고 파주기(sec)  12777 non-null  float64
 12  시도명          12777 non-null  int64  
dtypes: float64(10), int64(3)
memory usage: 1.3 MB


In [12]:
x_train.head()

,지점,일시,평균 풍속(m/s),평균기압(hPa),평균 상대습도(%),평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec),시도명
0,22190,5,5.872207,1010.8,71.53116,16.76661,16.0,2.6,1.6,4.0,6.3,9.1,2
1,22189,3,7.600000,1028.3,59.00000,11.30000,15.3,2.2,1.4,3.3,5.5,9.1,5
2,22105,8,2.000000,1010.8,82.00000,27.40000,27.1,2.9,1.8,3.9,9.2,10.7,3
3,22105,2,10.800000,1018.3,43.00000,2.60000,13.3,4.6,2.8,6.0,7.9,10.7,7
4,22105,6,7.000000,1008.1,86.00000,21.00000,20.2,1.9,1.1,2.4,5.1,5.8,6


In [13]:
x_test.head()

,지점,일시,평균 풍속(m/s),평균기압(hPa),평균 상대습도(%),평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec),시도명
0,22189,5,7.300000,1002.2,88.0,17.4,18.6,1.6,1.000000,2.5,4.6,5.8,2
1,22190,2,5.859536,1023.5,40.0,1.4,12.2,2.7,1.800000,4.2,7.4,9.1,3
2,22105,7,2.900000,1004.4,80.0,29.6,28.6,0.4,1.206909,0.7,3.0,4.0,5
3,22190,2,5.859536,1023.6,59.0,8.2,12.2,1.5,0.900000,2.2,8.1,9.1,5
4,22106,3,3.300000,1015.6,63.0,14.7,15.4,0.8,0.500000,1.1,7.2,8.0,2


In [14]:
# data scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x = sc.fit_transform(x_train)
test = sc.fit_transform(x_test)

In [15]:
# to tensor
x = torch.FloatTensor(np.array(x)).to(device)
y = torch.FloatTensor(np.array(y_train)).to(device)
test = torch.FloatTensor(np.array(test)).to(device)

In [16]:
# batch set
from torch.utils.data import DataLoader, TensorDataset

train_data = TensorDataset(x, y)
dataloader = DataLoader(train_data, batch_size = 50, shuffle = True, drop_last = True)

In [17]:
# training parameters
learning_rate = 0.01
training_epochs = 50

In [18]:
# seed value
torch.manual_seed(1)

# layer units
linear1 = nn.Linear(x.shape[1], 1024)
linear2 = nn.Linear(1024, 512)
linear3 = nn.Linear(512, 512)
linear4 = nn.Linear(512, 1024)
linear5 = nn.Linear(1024, 1)

# activation Function
relu = nn.ReLU()

In [19]:
# weight initializing
nn.init.xavier_uniform_(linear1.weight)
nn.init.xavier_uniform_(linear2.weight)
nn.init.xavier_uniform_(linear3.weight)
nn.init.xavier_uniform_(linear4.weight)
nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0410,  0.0394,  0.0115,  ..., -0.0055, -0.0223, -0.0198]],
       requires_grad=True)

In [20]:
# layers to model
model = nn.Sequential(linear1, relu,
                      linear2, relu,
                      linear3, relu,
                      linear4, relu,
                      linear5).to(device)

# loss function
loss = nn.MSELoss().to(device)

# optimizer
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [21]:
# model training

total_batch = len(dataloader)


for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in dataloader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
    
    if epoch % (5) == 0:
        print(epoch, avg_cost.item())

0 29652462.0
5 20069428.0
10 19331302.0
15 18992672.0
20 18380972.0
25 18341214.0
30 17915396.0
35 17821138.0
40 17840238.0
45 17763670.0


In [22]:
# prediction

with torch.no_grad():
    hypothesis = model(test)
    pred = hypothesis.cpu()
    result = pred.detach().numpy().astype(int)

# result

In [23]:
submit['price'] = result
submit

,id,price
0,10731,13132
1,2608,11429
2,14585,9909
3,4254,9695
4,6925,13531
...,...,...
5472,13176,10874
5473,16857,8849
5474,14220,6192
5475,6759,10335


In [24]:
submit.to_csv("defence.csv", index=False)